In [2]:
########## Find bootstrap confidence intervals for quantiles 

In [3]:
%%time
import numpy as np
import xarray as xr
from distributed import Client
import dask_jobqueue
import dask
#import daskdataframe as dd
import matplotlib.pyplot as plt
import glob
import netCDF4 as nc
import zarr
import nc_time_axis
import xskillscore as xs
import cartopy as cart
%run data_handler.ipynb
#np.set_printoptions(threshold=np.inf)

CPU times: user 17.6 s, sys: 2.34 s, total: 20 s
Wall time: 1min 38s


In [11]:
pi_yr0 = '1900'
pi_yr1 = '1930'
ic_yr0 = '2070'
ic_yr1 = '2100'
doy    = 211 # day_of_year
date   = 'Jul30'

In [4]:
def quantf(arr,qsteps=100,nboot=100):
    #Takes in a 1d-array,resamples it nboot times and
    #returns the quantile function for each iteration
    print(arr.shape)
    n            = arr.size
    qboots       = np.zeros((nboot,qsteps))
    quants       = np.arange(0,1,1/qsteps)
    for i in range(nboot):
        new_sample   = np.random.choice(arr,replace=True,size=n)
        qboots[i,:]  = np.quantile(new_sample,quants)
    
    return qboots   

In [5]:
%%time
### Test the function 'quantf'
arr1        = np.arange(1,20,1)
qtest       = quantf(arr1,10,5)
qtest

(19,)
CPU times: user 8.69 ms, sys: 3.08 ms, total: 11.8 ms
Wall time: 9.58 ms


array([[ 4. ,  5. ,  7.2,  9. ,  9.4, 11. , 12.6, 16.2, 17.4, 19. ],
       [ 3. ,  4.8,  5.6,  7. ,  7.6, 11. , 15.8, 16. , 16.4, 17.2],
       [ 1. ,  5. ,  6.6,  7.4, 10. , 10. , 11.8, 14.6, 15.4, 16.2],
       [ 1. ,  2. ,  3.6,  6.2,  8.2,  9. , 12.8, 14.2, 17.4, 19. ],
       [ 2. ,  2. ,  3.6,  5.4,  8.6, 11. , 13.8, 14.6, 15.8, 18. ]])

In [8]:
LAT        = 48.5
LON        = 236.25
cesm2_path = '/global/scratch/users/harsha/LENS/cesm2/tasmax/'
cvals      = '/global/scratch/users/harsha/LENS/cesm2/cvals/global/'
temp_path  = cvals+'temps/'
quant_path = cvals+'quantiles/'
quant_cis  = quant_path+'boot_cis/'
cvals_us   = '/global/scratch/users/harsha/LENS/cesm2/cvals/conus/'

In [ ]:
#job_extra = ['--qos=cf_lowprio','--account=ac_cumulus'] 
job_extra =['--qos=lr6_lowprio','--account=ac_cumulus','--constraint=lr6_m192'] 
cluster = dask_jobqueue.SLURMCluster(queue="lr6", cores=20, walltime='6:00:00', 
                local_directory='/global/scratch/users/harsha/dask_space/', 
                log_directory='/global/scratch/users/harsha/dask_space/', 
                job_extra_directives=job_extra, interface='eth0', memory="192GB") 
client  = Client(cluster) 
cluster.scale_up(5)
cluster

In [12]:
####### Get the temps ############
#Dictionary: g = global, nw= no-warming, w=warming, c= corrected, t= tasmax
tgnw_path = temp_path+'tnw'+date+'_'+pi_yr0+'_'+pi_yr1+'_global.zarr'
tgw_path  = temp_path+'tw'+date+'_'+ic_yr0+'_'+ic_yr1+'_global.zarr'
tgc_path  = temp_path+'twc'+date+'_'+ic_yr0+'_'+ic_yr1+'_global.zarr'
print(tgnw_path) 
print(tgw_path)
print(tgc_path)

/global/scratch/users/harsha/LENS/cesm2/cvals/global/temps/tnwJul30_1900_1930_global.zarr
/global/scratch/users/harsha/LENS/cesm2/cvals/global/temps/twJul30_2070_2100_global.zarr
/global/scratch/users/harsha/LENS/cesm2/cvals/global/temps/twcJul30_2070_2100_global.zarr


In [13]:
%%time
#Open zarr files
tgnw = xr.open_mfdataset(tgnw_path,engine='zarr')
tgw  = xr.open_mfdataset(tgw_path,engine='zarr')
tgc  = xr.open_mfdataset(tgc_path,engine='zarr')
tgw  = tgw.TREFHTMX
tgnw = tgnw.TREFHTMX
tgc  = tgc.TREFHTMX
tgw

CPU times: user 403 ms, sys: 78 ms, total: 481 ms
Wall time: 779 ms


<xarray.DataArray 'TREFHTMX' (lat: 192, lon: 288, mtime: 3100)>
dask.array<open_dataset-d8aa681214937eb6955504520d71f58cTREFHTMX, shape=(192, 288, 3100), dtype=float32, chunksize=(192, 288, 31), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member   (mtime) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    time     (mtime) object dask.array<chunksize=(3100,), meta=np.ndarray>
Dimensions without coordinates: mtime
Attributes:
    cell_methods:  time: maximum
    long_name:     Maximum reference height temperature over output period
    units:         K

In [15]:
%%time
######### Calculate bootstrap confidence intervals for quatiles
#Dict: g=global, b= bootstrap, w= warming, nw= no warming
nboot = 1000
tgnw  = tgnw.chunk(dict(mtime=-1))
tgw   = tgw.chunk(dict(mtime=-1))
tgc   = tgc.chunk(dict(mtime=-1))
######################################################
dask.config.set({"array.slicing.split_large_chunks": True})  #Split large chunks
#Resample using the function (reample_iterations) from xskill score
tgnwb   = xs.resample_iterations(tgnw,nboot,'mtime')
tgwb    = xs.resample_iterations(tgw,nboot,'mtime')
tgwcb   = xs.resample_iterations(tgc,nboot,'mtime')
tgwb

CPU times: user 1min 58s, sys: 512 ms, total: 1min 59s
Wall time: 1min 59s


<xarray.DataArray 'TREFHTMX' (lat: 192, lon: 288, mtime: 3100, iteration: 1000)>
dask.array<transpose, shape=(192, 288, 3100, 1000), dtype=float32, chunksize=(192, 288, 517, 1), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member     (mtime) int64 dask.array<chunksize=(3100,), meta=np.ndarray>
    time       (mtime) object dask.array<chunksize=(3100,), meta=np.ndarray>
  * iteration  (iteration) int64 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
Dimensions without coordinates: mtime
Attributes:
    cell_methods:  time: maximum
    long_name:     Maximum reference height temperature over output period
    units:         K

In [16]:
%%time
############ Compute quantiles and anomalies of quantiles ################
#Dict: a= anomaly, b= bootstrap, g= global, w = warming, nw= no warming
quants  = np.arange(0,1,0.001)
qgnwb   = tgnwb.chunk(dict(mtime=-1)).quantile(quants,dim='mtime')
qgwb    = tgwb.chunk(dict(mtime=-1)).quantile(quants,dim='mtime')
qgwcb   = tgwcb.chunk(dict(mtime=-1)).quantile(quants,dim='mtime')
qgwab   = qgwb - qgnwb
qgcab   = qgwcb - qgnwb
#Renaming to make sure I can take quantiles of quantiles
qgwab   = qgwab.rename({'quantile':'permilles'}) 
qgcab   = qgcab.rename({'quantile':'permilles'})
qgcab

CPU times: user 1.82 s, sys: 3.92 ms, total: 1.82 s
Wall time: 1.87 s


,Array,Chunk
Bytes,411.99 GiB,421.88 MiB
Shape,"(1000, 192, 288, 1000)","(1000, 192, 288, 1)"
Count,61204 Tasks,1000 Chunks
Type,float64,numpy.ndarray


In [17]:
### Get bootstrap confidence intervals of the quantiles.
#Dict: lb= lower bound, ub = upper bound, ca = anomaly corrected, wa= warm anomaly
qgwa_cilb = qgwab.chunk(dict(iteration=-1)).quantile(0.025,dim='iteration')
qgwa_ciub = qgwab.chunk(dict(iteration=-1)).quantile(0.975,dim='iteration')
qgca_cilb = qgcab.chunk(dict(iteration=-1)).quantile(0.025,dim='iteration')
qgca_ciub = qgcab.chunk(dict(iteration=-1)).quantile(0.975,dim='iteration')
qgca_ci   = xr.concat([qgca_cilb,qgca_ciub],dim='quantile')
qgwa_ci   = xr.concat([qgwa_cilb,qgwa_ciub],dim='quantile')
qgwa_ci

<xarray.DataArray 'TREFHTMX' (quantile: 2, permilles: 1000, lat: 192, lon: 288)>
dask.array<concatenate, shape=(2, 1000, 192, 288), dtype=float64, chunksize=(1, 1000, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * permilles  (permilles) float64 0.0 0.001 0.002 0.003 ... 0.997 0.998 0.999
  * quantile   (quantile) float64 0.025 0.975

In [ ]:
%%time
###### Save quantiles ########
qgwa_cipath = quant_cis+'qgwa_ci'+date+'_'+ic_yr0+'_'+ic_yr1+'.zarr'
qgca_cipath = quant_cis+'qgca_ci'+date+'_'+ic_yr0+'_'+ic_yr1+'.zarr'
################################
qgwa_ci.to_dataset().to_zarr(qgwa_cipath,mode='w')
qgca_ci.to_dataset().to_zarr(qgca_cipath,mode='w')